In [1]:
import pandas as pd
import requests
from datetime import datetime

In [20]:
team_coordinates = {
    "Ajaccio": {"latitude": 41.931081, "longitude": 8.776725},
    "Ajaccio GFCO": {"latitude": 41.92434, "longitude": 8.77273},
    "Bastia": {"latitude": 42.6514, "longitude": 9.442619},
    "Bordeaux": {"latitude": 44.829167, "longitude": -0.597778},
    "Evian Thonon Gaillard": {"latitude": 45.916497, "longitude": 6.118054},
    "Guingamp": {"latitude": 48.566285, "longitude": -3.164599},
    "Lens": {"latitude": 50.4264, "longitude": 2.8089},
    "Lille": {"latitude": 50.611883, "longitude": 3.130428},
    "Lorient": {"latitude": 47.748747, "longitude": -3.369367},
    "Lyon": {"latitude": 45.723889, "longitude": 4.832222},
    "Marseille": {"latitude": 43.269722, "longitude": 5.395833},
    "Monaco": {"latitude": 43.727606, "longitude": 7.415614},
    "Montpellier": {"latitude": 43.622222, "longitude": 3.811944},
    "Nantes": {"latitude": 47.255631, "longitude": -1.525375},
    "Nice": {"latitude": 43.723328, "longitude": 7.258756},
    "Paris SG": {"latitude": 48.841389, "longitude": 2.253056},
    "Reims": {"latitude": 49.246667, "longitude": 4.025},
    "Rennes": {"latitude": 48.107458, "longitude": -1.712839},
    "St Etienne": {"latitude": 45.460833, "longitude": 4.390278},
    "Sochaux": {"latitude": 47.512417, "longitude": 6.8112},
    "Toulouse": {"latitude": 43.583056, "longitude": 1.434167},
    "Valenciennes": {"latitude": 50.348131, "longitude": 3.533027},
    "Nancy": {"latitude": 48.6955, "longitude": 6.210687},
    "Dijon": {"latitude": 47.324383, "longitude": 5.068342},
    "Troyes": {"latitude": 48.307561, "longitude": 4.098458},
    "Auxerre": {"latitude": 47.786753, "longitude": 3.588664},
    "Brest": {"latitude": 48.402932, "longitude": -4.461694},
    "Caen": {"latitude": 49.179461, "longitude": -0.396767},
    "Angers": {"latitude": 47.460428, "longitude": 0.530803},      
    "Metz": {"latitude": 49.109814, "longitude": 6.159511},        
    "Amiens": {"latitude": 49.894208, "longitude": 2.263311},      
    "Strasbourg": {"latitude": 48.556036, "longitude": 7.752978},
    "Nimes": {"latitude": 43.810833, "longitude": 4.356389},       
}

In [11]:
def parse_date(csv_date):
    """
    Convert date from dd-mm-yy format to YYYY-MM-DD format.
    """
    # Parse the date from the CSV format (dd-mm-yy)
    date_obj = datetime.strptime(csv_date, "%d/%m/%y")
    # Format it as YYYY-MM-DD for the API
    return date_obj.strftime("%Y-%m-%d")

def fetch_weather_data(latitude, longitude, date):
    """
    Fetch weather data from the Open-Meteo API for a given date and location.
    """
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={date}&end_date={date}&hourly=temperature_2m,wind_speed_10m,precipitation&timezone=Europe/Berlin"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch weather data for {date}")
        return None

def extract_weather_metrics(weather_data):
    """
    Extract average temperature, wind speed, and total precipitation from weather data.
    """
    hourly = weather_data["hourly"]
    temperature = sum(hourly["temperature_2m"]) / len(hourly["temperature_2m"])
    wind_speed = sum(hourly["wind_speed_10m"]) / len(hourly["wind_speed_10m"])
    precipitation = sum(hourly["precipitation"])
    return temperature, wind_speed, precipitation

In [32]:
# Load the CSV file into a DataFrame
match_data = pd.read_csv("Datasets/Unprocessed/season-1819.csv")

# Add new columns for weather data
match_data["temperature"] = None
match_data["wind_speed"] = None
match_data["precipitation"] = None

In [33]:
# Loop through each row in the DataFrame
for index, row in match_data.iterrows():
    home_team = row["HomeTeam"]
    match_date = row["Date"]  # Date in dd-mm-yy format
    latitude = team_coordinates[home_team]["latitude"]
    longitude = team_coordinates[home_team]["longitude"]

    # Parse the date into YYYY-MM-DD format
    formatted_date = parse_date(match_date)

    # Fetch weather data
    weather_data = fetch_weather_data(latitude, longitude, formatted_date)
    if weather_data:
        temperature, wind_speed, precipitation = extract_weather_metrics(weather_data)
        # Update the DataFrame with weather data
        match_data.at[index, "temperature"] = temperature
        match_data.at[index, "wind_speed"] = wind_speed
        match_data.at[index, "precipitation"] = precipitation

# Save the updated DataFrame to a new CSV file
match_data.to_csv("Datasets/Processed/season-1819_with_weather.csv", index=False)

In [31]:
file_path = "Datasets/Processed/season-1819_with_weather.csv"
df = pd.read_csv(file_path)

# Find the column positions
# start_col = df.columns.get_loc("B365H")
# end_col = df.columns.get_loc("PSCA")

# Drop the columns in the range
df.drop(df.columns[22:63 + 1], axis=1, inplace=True)

# Save the modified file
df.to_csv("Datasets/Processed/season-1819_with_weather.csv", index=False)